In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time
import tensorflow as tf
# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections
from imblearn.over_sampling import SMOTE
# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.model_selection import KFold, StratifiedKFold
import pickle
import warnings
warnings.filterwarnings("ignore")
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, RobustScaler

In [2]:
df=pd.read_csv("final_list.csv")

In [3]:
df.head()

,Unnamed: 0,YEAR,T2M,T2MDEW,T2MWET,QV2M,RH2M,PRECTOTCORR,PS,WS10M,WD10M,WS50M,WD50M,Location,timestamp,lightning,year,month,day
0,0,2001,13.83,10.39,12.12,7.87,79.56,0.13,99.87,2.82,96.83,5.23,96.43,Banda,2001-01-01 00:00:00,0,2001,1,01 00:00:00
1,1,2001,13.73,10.64,12.19,8.00,81.44,0.12,99.83,2.64,107.90,4.69,107.05,Banda,2001-01-01 01:00:00,0,2001,1,01 01:00:00
2,2,2001,13.55,10.83,12.19,8.12,83.50,0.22,99.82,2.72,119.81,4.66,118.97,Banda,2001-01-01 02:00:00,0,2001,1,01 02:00:00
3,3,2001,13.37,10.97,12.17,8.18,85.19,0.10,99.80,3.02,129.96,4.91,129.44,Banda,2001-01-01 03:00:00,0,2001,1,01 03:00:00
4,4,2001,13.27,11.10,12.19,8.24,86.50,0.04,99.82,3.09,139.82,4.91,139.45,Banda,2001-01-01 04:00:00,0,2001,1,01 04:00:00


In [4]:
df.isnull().sum()

Unnamed: 0     0
YEAR           0
T2M            0
T2MDEW         0
T2MWET         0
QV2M           0
RH2M           0
PRECTOTCORR    0
PS             0
WS10M          0
WD10M          0
WS50M          0
WD50M          0
Location       0
timestamp      0
lightning      0
year           0
month          0
day            0
dtype: int64

In [5]:
# The classes are heavily skewed we need to solve this issue later.
print('No Lightning', round(df['lightning'].value_counts()[0]/len(df) * 100,2), '% of the dataset')
print('Lightning', round(df['lightning'].value_counts()[1]/len(df) * 100,2), '% of the dataset')

No Lightning 99.97 % of the dataset
Lightning 0.03 % of the dataset


In [6]:
df.drop(['timestamp','day','year','Location','Unnamed: 0','YEAR','month'], axis=1, inplace=True)

In [7]:
corr=df.select_dtypes(include=["int64","float64"])

In [8]:
corr=corr.corr()["lightning"][:-1]

In [9]:
corr.sort_values(ascending=False)

T2M            0.016072
T2MWET         0.014143
PRECTOTCORR    0.010481
T2MDEW         0.007289
QV2M           0.005776
WS10M          0.004454
WS50M         -0.001047
WD50M         -0.001385
WD10M         -0.001804
RH2M          -0.004721
PS            -0.005112
Name: lightning, dtype: float64

In [10]:
df

,T2M,T2MDEW,T2MWET,QV2M,RH2M,PRECTOTCORR,PS,WS10M,WD10M,WS50M,WD50M,lightning
0,13.83,10.39,12.12,7.87,79.56,0.13,99.87,2.82,96.83,5.23,96.43,0
1,13.73,10.64,12.19,8.00,81.44,0.12,99.83,2.64,107.90,4.69,107.05,0
2,13.55,10.83,12.19,8.12,83.50,0.22,99.82,2.72,119.81,4.66,118.97,0
3,13.37,10.97,12.17,8.18,85.19,0.10,99.80,3.02,129.96,4.91,129.44,0
4,13.27,11.10,12.19,8.24,86.50,0.04,99.82,3.09,139.82,4.91,139.45,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1507099,32.20,13.58,22.89,9.83,32.38,0.00,99.16,2.99,109.71,5.85,110.01,0
1507100,31.45,12.86,22.15,9.34,32.19,0.00,99.21,3.46,111.34,7.34,111.54,0
1507101,30.86,12.39,21.62,9.09,32.25,0.00,99.21,4.07,112.72,8.37,112.78,0
1507102,30.27,11.98,21.12,8.85,32.44,0.00,99.18,4.61,114.44,8.94,114.48,0


In [11]:

std_scaler = StandardScaler()
rob_scaler = RobustScaler()

df['sc_RH2M'] = rob_scaler.fit_transform(df['RH2M'].values.reshape(-1,1))
df['sc_QV2M'] = rob_scaler.fit_transform(df['QV2M'].values.reshape(-1,1))
df['sc_PRECTOTCORR'] = rob_scaler.fit_transform(df['PRECTOTCORR'].values.reshape(-1,1))
df['sc_T2M'] = rob_scaler.fit_transform(df['T2M'].values.reshape(-1,1))
df['sc_T2MDEW'] = rob_scaler.fit_transform(df['T2MDEW'].values.reshape(-1,1))
df['sc_T2MWET'] = rob_scaler.fit_transform(df['T2MWET'].values.reshape(-1,1))
df['sc_PS'] = rob_scaler.fit_transform(df['PS'].values.reshape(-1,1))
df['sc_WS10M'] = rob_scaler.fit_transform(df['WS10M'].values.reshape(-1,1))
df['sc_WD10M'] = rob_scaler.fit_transform(df['WD10M'].values.reshape(-1,1))
df['sc_WS50M'] = rob_scaler.fit_transform(df['WS50M'].values.reshape(-1,1))
df['sc_WD50M'] = rob_scaler.fit_transform(df['WD50M'].values.reshape(-1,1))
df.drop(['RH2M','QV2M','PRECTOTCORR', 'T2M','T2MDEW','T2MWET','PS','WS10M','WD10M','WS50M','WD50M'], axis=1, inplace=True)

In [12]:
df.head()

,lightning,sc_RH2M,sc_QV2M,sc_PRECTOTCORR,sc_T2M,sc_T2MDEW,sc_T2MWET,sc_PS,sc_WS10M,sc_WD10M,sc_WS50M,sc_WD50M
0,0,0.627724,-0.060606,13.0,-1.156584,-0.068220,-0.638332,0.568047,0.046243,-0.727040,0.314103,-0.738600
1,0,0.666736,-0.049959,12.0,-1.165480,-0.054354,-0.632719,0.544379,-0.057803,-0.671484,0.141026,-0.685028
2,0,0.709483,-0.040131,22.0,-1.181495,-0.043816,-0.632719,0.538462,-0.011561,-0.611713,0.131410,-0.624899
3,0,0.744553,-0.035217,10.0,-1.197509,-0.036051,-0.634322,0.526627,0.161850,-0.560775,0.211538,-0.572084
4,0,0.771737,-0.030303,4.0,-1.206406,-0.028841,-0.632719,0.538462,0.202312,-0.511292,0.211538,-0.521590


In [13]:
df['lightning'].value_counts()[1]

414

In [14]:
df['lightning'].value_counts()[0]

1506690

In [15]:
y= df['lightning']

In [16]:
X=df.drop(['lightning'],axis='columns')

In [17]:
smote=SMOTE(sampling_strategy='minority')

In [18]:
x_sm,y_sm=smote.fit_resample(X,y)

In [19]:
y_sm.value_counts()

0    1506690
1    1506690
Name: lightning, dtype: int64

In [20]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_sm,y_sm,test_size=0.2,random_state=15,stratify=y_sm)

In [ ]:
#neural netwrok model


model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(11,)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')  # Binary classification, so use sigmoid activation
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    epochs=100)  # Split a portion of the training data for validation

Epoch 1/100
11491/75335 [===>..........................] - ETA: 57s - loss: 0.4125 - accuracy: 0.8260

In [46]:
history = model.fit(x_train, y_train,
                    epochs=100)  # Split a portion of the training data for validation


Epoch 1/100
75335/75335 [==============================] - 75s 987us/step - loss: 0.3091 - accuracy: 0.8720
Epoch 2/100
75335/75335 [==============================] - 79s 1ms/step - loss: 0.2620 - accuracy: 0.8961
Epoch 3/100
75335/75335 [==============================] - 79s 1ms/step - loss: 0.2503 - accuracy: 0.9016
Epoch 4/100
75335/75335 [==============================] - 80s 1ms/step - loss: 0.2429 - accuracy: 0.9054ETA: 53s - loss: 0.2446 - accura - ETA: 5 - ETA: 51s - loss: 0.2446 - accuracy: 0.90 - ETA: 51s - loss: 0.2446 - a - ETA: 50s - loss: - ETA: 49s - loss: 0.2445 - accuracy: 0.90 - ETA: 49s  - ETA: 47s - loss:  - ETA: 3s - loss: 0.2430 - accu - ETA: 0s
Epoch 5/100
75335/75335 [==============================] - 80s 1ms/step - loss: 0.2382 - accuracy: 0.9075 0s - loss: 0.2382 - ac
Epoch 6/100
75335/75335 [==============================] - 80s 1ms/step - loss: 0.2339 - accuracy: 0.9096
Epoch 7/100
75335/75335 [==============================] - 81s 1ms/step - loss: 0.2313 - 

In [47]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test Accuracy:", test_accuracy)


18834/18834 [==============================] - 10s 513us/step - loss: 0.1376 - accuracy: 0.9524
Test Accuracy: 0.9523757100105286


In [48]:
predictions = model.predict(x_test)
y_pred=np.round(predictions)

In [50]:
print("report \n",classification_report(y_test,y_pred))

report 
               precision    recall  f1-score   support

           0       1.00      0.91      0.95    301338
           1       0.91      1.00      0.95    301338

    accuracy                           0.95    602676
   macro avg       0.96      0.95      0.95    602676
weighted avg       0.96      0.95      0.95    602676



In [52]:
from keras.models import load_model
model.save('lightening_classfier.h5')


In [56]:
loaded_model = load_model('lightening_classfier.h5')
x_demo=np.asarray([13.83,10.39,12.12,7.87,79.56,0.13,99.87,2.82,96.83,5.23,96.43])
y_pred = loaded_model.predict(x_demo.reshape(1, -1))

In [58]:
print(y_pred)

[[0.]]
